# ML : Regresión

## Objetivos académicos

1. **Comprender los fundamentos de la regresión en Machine Learning**, diferenciando entre tareas de regresión y clasificación, y reconociendo aplicaciones prácticas de la regresión.

2. **Aplicar y comparar distintos modelos de regresión** (regresión lineal, árbol de decisión y random forest) para predecir valores numéricos a partir de datos reales.

3. **Evaluar el desempeño de modelos de regresión** utilizando métricas como el error cuadrático medio (ECM) y la raíz del error cuadrático medio (RECM), interpretando sus resultados.

4. **Optimizar modelos mediante el ajuste de hiperparámetros con GridSearchCV**, identificando los parámetros clave y comprendiendo su impacto en la precisión y generalización del modelo.

## 🟢 1. ¿Qué es una tarea de regresión?

En Machine Learning, una **tarea de regresión** consiste en predecir un **valor numérico continuo**. A diferencia de la clasificación (donde elegimos una categoría como “benigno” o “maligno”), aquí el objetivo es predecir una cantidad: un número real.

---

### 🍡 Ejemplos del mundo real

* ¿Cuál será el precio de esta vivienda? 🏡  
* ¿Cuántas bicicletas se rentarán mañana? 🚴‍♂️  
* ¿Cuánto ganará este cliente el próximo mes? 💵  
* ¿Qué temperatura habrá la semana que viene? 🌡️

En todos estos casos, no buscamos etiquetar, sino **estimar una cantidad**. Y eso es regresión.

---

### 🌼 Diferencia clave: clasificación vs regresión

| Tipo de problema | Objetivo                  | Ejemplo                         |
|------------------|---------------------------|----------------------------------|
| Clasificación    | Predecir una categoría    | ¿El tumor es benigno o maligno? |
| Regresión        | Predecir un número real   | ¿Cuánto cuesta una casa?        |

> En resumen: **Clasificación = ¿Qué clase? / Regresión = ¿Cuánto?**


## 🔵 2. Nuestro reto: predecir el precio medio de una vivienda en California 🏡

Vamos a usar un dataset real llamado **California Housing**, que contiene información sobre viviendas y distritos en California. Cada fila representa una zona residencial, y las columnas contienen datos agregados de esa zona.

---

### 📊 Características del dataset

| Columna     | Descripción                                       |
|-------------|---------------------------------------------------|
| `MedInc`    | Ingreso medio del distrito (en decenas de miles) |
| `HouseAge`  | Antigüedad media de las viviendas (en años)      |
| `AveRooms`  | Número promedio de habitaciones por casa         |
| `AveOccup`  | Promedio de personas por vivienda                |
| `Latitude`  | Latitud del distrito                             |
| `Longitude` | Longitud del distrito                            |
| `MedHouseVal` | Valor medio de las casas (*target*)           |


### 📥 Cargar datos y librerías

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
import joblib

In [ ]:
# Cargar el dataset
housing = fetch_california_housing(as_frame=True)
df = housing.frame

# Separar features y target
X = df.drop(columns=['MedHouseVal'])
y = df['MedHouseVal']

## ✂️ Preparación de los datos

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=42)

### 🍥 ¿Cuál es nuestro objetivo?

Vamos a construir un modelo que, a partir de variables como el ingreso medio y el número de habitaciones, sea capaz de predecir el **valor promedio de una vivienda** en esa zona.

* **Entrada (`X`)**: características numéricas como `MedInc`, `HouseAge`, etc.
* **Salida (`y`)**: `MedHouseVal`, el precio medio.


## 🟡 3. ¿Cómo se mide el error en regresión?

En problemas de clasificación usamos métricas como *accuracy*, *precision* o *recall* porque las respuestas correctas eran categorías: sí/no, A/B, 0/1.

Pero en regresión, **nunca esperamos acertar exactamente el valor real**, sino estar lo más cerca posible.

Por eso, las métricas de regresión se enfocan en medir **qué tan lejos** están nuestras predicciones de los valores reales.


### 🍥 ¿Qué es el *error*?

Es simplemente la diferencia entre lo que predice el modelo y el valor correcto:

$$
\text{Error} = \hat{y}_i - y_i
$$

Donde:
* $(\hat{y}_i)$: es el valor predicho por el modelo para la observación $(i)$ 
* $(y_i)$: es el valor real


### 📄 Error Cuadrático Medio (ECM / MSE)

El ECM mide el promedio de los cuadrados de los errores:

$$
\text{ECM} = \frac{1}{n} \sum_{i=1}^{n} (\hat{y}_i - y_i)^2
$$

**Interpretación:**
- Penaliza más los errores grandes (porque los eleva al cuadrado).
- Siempre es un número positivo.
- Cuanto más bajo, mejor.

> Pero sus unidades son “dólares al cuadrado” si estás prediciendo precios, lo cual no es tan intuitivo.


### 🗂️ Raíz del ECM (RECM / RMSE)

Para volver a las mismas unidades del valor original (por ejemplo, dólares), tomamos la raíz cuadrada del ECM. A esto se le llama:

$$
\text{RECM} = \sqrt{\text{ECM}}
$$

**Ejemplo práctico:**

- Si el RECM es `0.45`, significa que en promedio, nos equivocamos unos **$45,000 dólares** (recordando que los valores están en cientos de miles).


## 🔴 4. Árbol de decisión para regresión 🌳

Ya vimos que un árbol de decisión funciona como una serie de preguntas del tipo:

> “¿El ingreso medio es mayor a 5.5?”  
> “¿La antigüedad de la vivienda es menor a 20 años?”

En clasificación, cada hoja del árbol contenía una **clase** (por ejemplo, “maligno”).  
En regresión, cada hoja contiene un **valor promedio**, calculado con los ejemplos que caen en ese nodo.

---

### 🧠 Intuición del árbol regresor

Un **árbol de regresión** divide el espacio de los datos en regiones donde las predicciones son constantes. Es decir:

- Cada región termina con un promedio de los valores verdaderos de entrenamiento.
- Es como decir: *“si el ingreso medio está entre 2 y 4 y la ubicación es tal, entonces predice \$120,000”*.

Esto lo convierte en un modelo muy fácil de interpretar y visualizar.


In [ ]:
tree_reg = DecisionTreeRegressor(max_depth=4, random_state=42)
tree_reg.fit(X_train, y_train)
preds_tree = tree_reg.predict(X_valid)

mse = mean_squared_error(y_valid, preds_tree)
rmse = mse ** 0.5
print("ECM (Árbol):", mse)
print("RECM (Árbol):", rmse)

In [ ]:
plt.figure(figsize=(6,6))
plt.scatter(y_valid, preds_tree, alpha=0.5)
plt.plot([y_valid.min(), y_valid.max()], [y_valid.min(), y_valid.max()], '--r')
plt.xlabel("Valor real")
plt.ylabel("Predicción")
plt.title("Árbol de decisión: Real vs Predicción")
plt.grid(True)
plt.show()

> Si los puntos están muy cerca de la línea roja diagonal (real = predicción), ¡vamos por buen camino!



## 🟣 5. Regresión Lineal 📈

La **regresión lineal** es uno de los modelos más simples y potentes para predecir valores numéricos. A pesar de su simplicidad, muchas veces es sorprendentemente efectiva y es un excelente punto de partida en cualquier problema de regresión.

---

### 🧾 ¿Qué significa “lineal”?

Significa que el modelo intenta **ajustar una línea (o un plano, o un hiperplano)** que se aproxime lo mejor posible a los datos. Esa línea sigue esta fórmula:

$$
\hat{y} = b_0 + b_1 x_1 + b_2 x_2 + \cdots + b_n x_n
$$

**Donde:**
- $\hat{y}$ es el valor predicho.
- $x_1, x_2, ..., x_n$ son las características (por ejemplo, ingreso, habitaciones, edad).
- $b_0$ es la intersección con el eje y.
- $b_1, ..., b_n$ son los coeficientes o **pesos** que determinan la influencia de cada característica.

---

### 🧠 Intuición

Cada característica “empuja” el valor final hacia arriba o hacia abajo.

Por ejemplo:

- Si `MedInc` tiene un peso positivo, a mayor ingreso medio, mayor será el precio de la vivienda.
- Si `HouseAge` tiene un peso negativo, a mayor antigüedad, menor el valor.


In [ ]:
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
preds_lin = lin_reg.predict(X_valid)

mse_lin = mean_squared_error(y_valid, preds_lin)
rmse_lin = mse_lin ** 0.5
print("ECM (Lineal):", mse_lin)
print("RECM (Lineal):", rmse_lin)

In [ ]:
plt.figure(figsize=(6,6))
plt.scatter(y_valid, preds_lin, alpha=0.5, color="orange")
plt.plot([y_valid.min(), y_valid.max()], [y_valid.min(), y_valid.max()], '--r')
plt.xlabel("Valor real")
plt.ylabel("Predicción")
plt.title("Regresión lineal: Real vs Predicción")
plt.grid(True)
plt.show()

> Una buena regresión debe concentrar sus puntos cerca de la línea roja. Si vemos mucho “abanico”, probablemente necesitamos un modelo más flexible.

## 🟤 6. Random Forest para regresión 🌲

Ya trabajamos con:

* Árbol de decisión, que es fácil de interpretar, pero puede sobreajustarse.
* Regresión lineal, que es simple y rápida, pero puede no capturar relaciones complejas.

Ahora presentamos un modelo que combina **lo mejor de ambos mundos**: el **Random Forest**.

---

### 🧠 ¿Qué es un Random Forest?

Un **bosque aleatorio** es un conjunto de muchos árboles de decisión entrenados con **subconjuntos aleatorios** de los datos y las características.

> Cada árbol hace su predicción, y el resultado final es el **promedio** de todas las predicciones (en regresión) o el **voto mayoritario** (en clasificación).

---

### 🍥 Ventajas del Random Forest

✅ Generaliza bien: no se queda atrapado en los datos de entrenamiento.  
✅ Maneja relaciones no lineales.  
✅ No necesita mucho ajuste.  
✅ Resistente al ruido.

---

### ⚙️ Hiperparámetros clave

* `n_estimators`: número de árboles en el bosque.
* `max_depth`: profundidad máxima de cada árbol (opcional).
* `random_state`: semilla para reproducibilidad.


In [ ]:
rf_reg = RandomForestRegressor(n_estimators=100, random_state=42)
rf_reg.fit(X_train, y_train)
preds_rf = rf_reg.predict(X_valid)

mse_rf = mean_squared_error(y_valid, preds_rf)
rmse_rf = mse_rf ** 0.5
print("ECM (RF):", mse_rf)
print("RECM (RF):", rmse_rf)

In [ ]:
plt.figure(figsize=(6,6))
plt.scatter(y_valid, preds_rf, alpha=0.5, color="green")
plt.plot([y_valid.min(), y_valid.max()], [y_valid.min(), y_valid.max()], '--r')
plt.xlabel("Valor real")
plt.ylabel("Predicción")
plt.title("Random Forest: Real vs Predicción")
plt.grid(True)
plt.show()

## 📊 Comparación de modelos

In [ ]:
modelos = ['Árbol', 'Lineal', 'Random Forest']
mse_scores = [mse, mse_lin, mse_rf]
rmse_scores = [rmse, rmse_lin, rmse_rf]

for modelo, mse_val, rmse_val in zip(modelos, mse_scores, rmse_scores):
    print(f"{modelo}")
    print(f"  ECM: {mse_val:.4f}")
    print(f"  RECM: {rmse_val:.4f}\n")

In [ ]:
x = range(len(modelos))
plt.figure(figsize=(10,5))
plt.bar(x, rmse_scores, tick_label=modelos, color=["skyblue", "orange", "green"])
plt.ylabel("RECM")
plt.title("Comparación de modelos - RECM")
plt.ylim(0, max(rmse_scores) + 0.2)
plt.grid(axis='y')
plt.xticks(rotation=15)
plt.show()

### 🧠 ¿Cómo interpretar los resultados?

* El modelo con **RECM más bajo** es el que se **equivoca menos** en promedio.
* Si el **Random Forest** obtiene el mejor resultado (lo más común), puedes justificar su elección aunque no sea tan interpretable como la regresión lineal.

> Elegir un modelo no siempre es solo por precisión: a veces se valora también la **interpretabilidad**, **velocidad** o **robustez** según el contexto.


## 💾 8. Guardar el modelo final

Una vez que tenemos un modelo bien evaluado (por ejemplo, el **Random Forest** que obtuvo el menor RECM), podemos **guardarlo en un archivo** para:

* Reutilizarlo más tarde sin tener que reentrenar.
* Integrarlo en una aplicación real.
* Compartirlo con otros miembros del equipo.
* Versionarlo como parte de un flujo de trabajo reproducible.


In [ ]:
joblib.dump(rf_reg, "modelo_valor_vivienda.joblib")

# Cargar el modelo desde archivo
modelo_cargado = joblib.load("modelo_valor_vivienda.joblib")

# Usar el modelo cargado para hacer predicciones
nuevas_predicciones = modelo_cargado.predict(X_valid)

## 🧩 9. Ajuste de hiperparámetros con GridSearch

Para mejorar el rendimiento de un modelo como Random Forest, es fundamental ajustar sus hiperparámetros, es decir, aquellos valores que controlan el comportamiento del algoritmo (por ejemplo, el número de árboles, la profundidad máxima, etc.). Una técnica muy utilizada es **GridSearch**, que consiste en probar de manera sistemática todas las combinaciones posibles de un conjunto de valores para los hiperparámetros seleccionados. Usando `GridSearchCV` de `scikit-learn`, podemos automatizar este proceso: el método entrena y valida el modelo con cada combinación, utilizando validación cruzada, y al final nos indica cuál es la mejor configuración según la métrica elegida (por ejemplo, el menor RECM). Así, GridSearch nos ayuda a encontrar el modelo más preciso de forma eficiente y reproducible.



### 🔧 Principales hiperparámetros para ajustar en Random Forest

Al utilizar GridSearch para optimizar un modelo Random Forest, es importante considerar los hiperparámetros más relevantes que influyen en su desempeño. Entre los principales se encuentran:

- **`n_estimators`**: número de árboles en el bosque. Más árboles suelen mejorar la precisión, pero aumentan el tiempo de cómputo.
- **`max_depth`**: profundidad máxima de cada árbol. Limitarla ayuda a evitar el sobreajuste.
- **`min_samples_split`**: número mínimo de muestras necesarias para dividir un nodo interno.
- **`min_samples_leaf`**: número mínimo de muestras requeridas en una hoja.
- **`max_features`**: número máximo de características consideradas al buscar la mejor división en cada nodo.
- **`bootstrap`**: indica si se usan muestras con reemplazo para construir los árboles.
- **`random_state`**: semilla para asegurar la reproducibilidad de los resultados.

Ajustar estos hiperparámetros permite encontrar el equilibrio óptimo entre precisión, generalización y eficiencia computacional del modelo.

In [ ]:

# Definir el diccionario de hiperparámetros a probar
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 3],
    'max_features': ['log2', 'sqrt']
}

# Crear el modelo base
rf = RandomForestRegressor(random_state=42)

# Configurar GridSearchCV
grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=3,  # 3-fold cross-validation
    scoring='neg_root_mean_squared_error',  # Para minimizar el RECM
    n_jobs=-1,  # Usar todos los núcleos disponibles
    verbose=2
)

# Ajustar el modelo
grid_search.fit(X_train, y_train)

# Mostrar los mejores hiperparámetros encontrados
print("Mejores hiperparámetros:", grid_search.best_params_)
print("Mejor RECM (validación cruzada):", -grid_search.best_score_)

In [ ]:
best_gridsearch = grid_search.best_estimator_
predict_gridsearch = best_gridsearch.predict(X_valid)
mse_rf = mean_squared_error(y_valid, predict_gridsearch)
rmse_rf = mse_rf ** 0.5
print("ECM (RF):", mse_rf)
print("RECM (RF):", rmse_rf)


In [ ]:
plt.figure(figsize=(6,6))
plt.scatter(y_valid, predict_gridsearch, alpha=0.5, color="green")
plt.plot([y_valid.min(), y_valid.max()], [y_valid.min(), y_valid.max()], '--r')
plt.xlabel("Valor real")
plt.ylabel("Predicción")
plt.title("Random Forest GridSearch: Real vs Predicción")
plt.grid(True)
plt.show()

# En resumen

1. **Regresión :**  Es una tarea donde el objetivo es predecir valores numéricos continuos, a diferencia de la clasificación que predice categorías.

2. **Métricas de evaluación:** : El error cuadrático medio (ECM) y la raíz del error cuadrático medio (RECM) son métricas clave para medir la precisión de los modelos de regresión.

3. **Ajuste de hiperparámetros:** :El uso de GridSearchCV permite encontrar la mejor combinación de hiperparámetros para mejorar el rendimiento del modelo, especialmente en random forest.

4. **Interpretación y visualización de resultados:** : Analizar gráficamente las predicciones frente a los valores reales ayuda a identificar el ajuste del modelo y posibles áreas de mejora.

